<a href="https://colab.research.google.com/github/theshivamgiri/Smart_Irrigation_System-AquaSmartAgri/blob/main/UI_for_farmers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
import requests
import pandas as pd
import numpy as np

# URL for ThingSpeak channel data (replace 'YOUR_CHANNEL_ID' and 'YOUR_API_KEY')
url = 'https://api.thingspeak.com/channels/2648719/feeds.json?api_key=T9BTHEE3NKREFMUD&results=1000'

response = requests.get(url)
data = response.json()

# Convert JSON data to a DataFrame
feeds = data['feeds']
df = pd.DataFrame(feeds)
df['created_at'] = pd.to_datetime(df['created_at'])
df['field1'] = pd.to_numeric(df['field1'], errors='coerce')  # Temperature
df['field2'] = pd.to_numeric(df['field2'], errors='coerce')  # Humidity

# Rename columns for clarity
df.rename(columns={'field1': 'Temperature', 'field2': 'Humidity'}, inplace=True)

print(df.tail())

                   created_at  entry_id  Temperature  Humidity
99  2024-11-18 11:38:14+00:00       100         30.4      11.0
100 2024-11-18 11:38:39+00:00       101         29.4      11.0
101 2024-11-18 11:39:01+00:00       102         27.2      11.0
102 2024-11-18 11:39:23+00:00       103         21.9      27.0
103 2024-11-18 11:39:45+00:00       104          9.0      27.0


In [124]:
# Drop rows with NaN values
df.dropna(subset=['Temperature', 'Humidity'], inplace=True)

# Rename columns for clarity
df.rename(columns={'field1': 'Temperature', 'field2': 'Humidity'}, inplace=True)

# Add features such as time of day or average temperature/humidity for better modeling
df['hour'] = df['created_at'].dt.hour

# Example target variable: suggest whether irrigation is needed
df['Irrigation_Need'] = np.where((df['Temperature'] > 30) & (df['Humidity'] < 50), 1, 0)  # Simplified rule

In [125]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Prepare data for training
X = df[['Temperature', 'Humidity', 'hour']]
y = df['Irrigation_Need']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=90)

# Train a RandomForest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')


Accuracy: 0.8571428571428571


In [126]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

In [127]:
import pandas as pd
from IPython.core.display import display, HTML

import time

# Extract the last row values
last_row = df.iloc[-1]
temp = last_row['Temperature']
humidity = last_row['Humidity']
hour = last_row['hour']

# Display the extracted data in HTML with Hindi labels
# future scope - more local language & voice assitance
html_code = f"""
<!DOCTYPE html>
<html lang="hi">
<head>
    <meta charset="UTF-8">
    <title>स्मार्ट सिंचाई प्रणाली</title>
</head>
<body>
    <h1>स्मार्ट सिंचाई सहायता प्रणाली</h1>
    <p><strong>तापमान (°C):</strong> {temp}</p>
    <p><strong>नमी (%):</strong> {humidity}</p>
    <p><strong>समय (घंटे):</strong> {hour}</p>
</body>
</html>
"""
display(HTML(html_code))

# Assuming 'model' is already loaded and your dataset is in 'df'
def predict_last_row(df, model, delay=9):
    # Extract the last row of the DataFrame
    last_row = df.iloc[-1]
    temp = last_row['Temperature']
    humidity = last_row['Humidity']
    hour = last_row['hour']

    # Create DataFrame for the input
    input_data = pd.DataFrame([[temp, humidity, hour]], columns=['Temperature', 'Humidity', 'hour'])

    # Make a prediction
    prediction = model.predict(input_data)

    # Introduce delay
    #time.sleep(delay)

    result = "सिंचाई की आवश्यकता है" if prediction[0] == 1 else "सिंचाई की आवश्यकता नहीं है"
    html_output = f"""
    <div style="font-size: 27px; font-weight: bold; color: red ; text-align: center;">
        {result}
    </div>
    """
    display(HTML(html_output))
    return


# Example usage:
# Assuming 'df' is your DataFrame and 'model' is your trained model
print(predict_last_row(df, model))

from IPython.display import IFrame, display

# URLs for the four charts
chart_urls = [
    'https://thingspeak.mathworks.com/channels/2648719/charts/1?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&title=Temperature+Chart&type=line',
    'https://thingspeak.mathworks.com/channels/2648719/charts/2?bgcolor=%23ffffff&color=%2300aaff&dynamic=true&results=60&title=Humidity+Chart&type=line',
    'https://thingspeak.mathworks.com/channels/2648719/widgets/916916',
    'https://thingspeak.mathworks.com/channels/2648719/widgets/916917'
]

# Display all charts in separate IFrames
for url in chart_urls:
    display(IFrame(url, width=540, height=270))



None
